In [ ]:
NVS_namespace = "controller"

    # timeValue_t workFromHomeWakeUp = {  7, 00 };
    # timeValue_t workFromHomeSleep  = { 21, 30 };

    # timeValue_t workAtOfficeWakeUp = {  6, 30 };
    # timeValue_t workAtOfficeGoOut  = {  7, 15 };
    # timeValue_t workAtOfficeComeIn = { 17, 15 };
    # timeValue_t workAtOfficeSleep  = { 21, 30 };

    # timeValue_t weekendWakeUp      = {  9, 00 };
    # timeValue_t weekendSleep       = { 22, 00 };

#  0               1               2          3       4
# {dtWorkFromHome, dtWorkAtOffice, dtWeekend, dtAway, dtAuto};

# Key, type, default, variable
# Keys were truncated to 15 characters, since this is maximum key length in ESP32 NVS
persistent_variables = [
  [ "workFromHmeWkUp", "int",       "7*60+ 0", "workFromHomeWakeUp.minutesSinceMidnight"],
  [ "workFromHmeSlep", "int",       "21*60+30", "workFromHomeSleep.minutesSinceMidnight"],
  [ "workAtOffWakeUp", "int",       "6*60+30", "workAtOfficeWakeUp.minutesSinceMidnight"],
  [ "workAtOffGoOut",  "int",       "7*60+15", "workAtOfficeGoOut.minutesSinceMidnight"],
  [ "workAtOffComeIn", "int",       "17*60+15", "workAtOfficeComeIn.minutesSinceMidnight"],
  [ "workAtOffSleep",  "int",       "21*60+30", "workAtOfficeSleep.minutesSinceMidnight"],
  [ "weekendWakeUp",   "int",       "9*60+30", "weekendWakeUp.minutesSinceMidnight"],
  [ "weekendSleep",    "int",       "22*60+ 0", "weekendSleep.minutesSinceMidnight"],
  [ "regularWeek0",    "dayType_t",        "2", "regularWeek[0]"], # Sunday = dtWeekend
  [ "regularWeek1",    "dayType_t",        "1", "regularWeek[1]"], # Monday = dtWorkAtOffice
  [ "regularWeek2",    "dayType_t",        "0", "regularWeek[2]"], # Tuesday = dtWorkFromHome
  [ "regularWeek3",    "dayType_t",        "1", "regularWeek[3]"], # Wednesday = dtWorkAtOffice
  [ "regularWeek4",    "dayType_t",        "1", "regularWeek[4]"], # Thursday = dtWorkAtOffice
  [ "regularWeek5",    "dayType_t",        "0", "regularWeek[5]"], # Friday = dtWorkFromHome
  [ "regularWeek6",    "dayType_t",        "2", "regularWeek[6]"], # Saturday = dtWeekend
  [ "highTemp",        "float",         "19.5", "highTemp"],
  [ "lowTemp",         "float",           "17", "lowTemp"],
  [ "overrideToday",   "dayType_t",        "4", "overrideToday"],      # dtAuto
  [ "overrideTomrrw",  "dayType_t",        "4", "overrideTomorrow"],   # dtAuto
  [ "overrideMltpl",   "dayType_t",        "4", "overrideMultiple"],   # dtAuto
  [ "overrideMultCnt", "int",              "0", "overrideMultipleCount"],
  [ "multipleForever", "bool",         "false", "multipleForever"],
  [ "sensorOffset",    "float",         "-3.5", "sensorOffset"] ]

# Check if length is not exceeded
for key, typ, dft, var in persistent_variables:
    if len(key)>15:
      print(len(key), key)

code = f"""#include <Preferences.h>

bool ControllerData_t::saveSettingsNVS() {{
  Preferences pref;
  if (!pref.begin("{NVS_namespace}", false)) {{
    Serial.println("Failed to open NVS for writing");
    return false;
  }}
""".split('\n')

for key, typ, dft, var in persistent_variables:
  if(typ=="int" or typ=="dayType_t"):
    code.append(f'  pref.putInt("{key}", (int){var});')
  elif(typ=="float"):
    code.append(f'  pref.putFloat("{key}", {var});')
  elif(typ=="bool"):
    code.append(f'  pref.putBool("{key}", {var});')
  else:
    print(f"key: {key} has unknown type {typ} and var {var}")

code.extend("""  pref.end();
  settingsChanged = false;
  return true;
}
""".split('\n'))

code.extend(f"""bool ControllerData_t::loadSettingsNVS() {{
  Preferences pref;
  if (!pref.begin("{NVS_namespace}", true)) {{   // true = read-only
    Serial.println("Failed to open NVS for reading");
    return false;
  }}
""".split('\n'))

for key, typ, dft, var in persistent_variables:
  if(typ=="int"):
    code.append(f'  {var} = pref.getInt("{key}", {dft});')
  elif(typ=="dayType_t"):
    code.append(f'  {var} = (dayType_t)pref.getInt("{key}", {dft});')
  elif(typ=="float"):
    code.append(f'  {var} = pref.getFloat("{key}", {dft});')
  elif(typ=="bool"):
    code.append(f'  {var} = pref.getBool("{key}", {dft});')
  else:
    print(f"key: {key} has unknown type {typ} and var {var}")

code.extend("""  pref.end();
  return true;
}""".split('\n'))

print('\n'.join(code))
import pyperclip
pyperclip.copy('\n'.join(code))





#include <Preferences.h>

bool ControllerData_t::saveSettingsNVS() {
  Preferences pref;
  if (!pref.begin("controller", false)) {
    Serial.println("Failed to open NVS for writing");
    return false;
  }

  pref.putInt("workFromHmeWkUp", (int)workFromHomeWakeUp.minutesSinceMidnight);
  pref.putInt("workFromHmeSlep", (int)workFromHomeSleep.minutesSinceMidnight);
  pref.putInt("workAtOffWakeUp", (int)workAtOfficeWakeUp.minutesSinceMidnight);
  pref.putInt("workAtOffGoOut", (int)workAtOfficeGoOut.minutesSinceMidnight);
  pref.putInt("workAtOffComeIn", (int)workAtOfficeComeIn.minutesSinceMidnight);
  pref.putInt("workAtOffSleep", (int)workAtOfficeSleep.minutesSinceMidnight);
  pref.putInt("weekendWakeUp", (int)weekendWakeUp.minutesSinceMidnight);
  pref.putInt("weekendSleep", (int)weekendSleep.minutesSinceMidnight);
  pref.putInt("regularWeek0", (int)regularWeek[0]);
  pref.putInt("regularWeek1", (int)regularWeek[1]);
  pref.putInt("regularWeek2", (int)regularWeek[2]);
  pref.putInt("re

```cpp
#include <Preferences.h>

bool ControllerData_t::saveSettingsNVS() {
  Preferences pref;
  if (!pref.begin("controller", false)) {
    Serial.println("Failed to open NVS for writing");
    return false;
  }

  pref.putInt("workFromHmeWkUp", (int)workFromHomeWakeUp.minutesSinceMidnight);
  pref.putInt("workFromHmeSlep", (int)workFromHomeSleep.minutesSinceMidnight);
  pref.putInt("workAtOffWakeUp", (int)workAtOfficeWakeUp.minutesSinceMidnight);
  pref.putInt("workAtOffGoOut", (int)workAtOfficeGoOut.minutesSinceMidnight);
  pref.putInt("workAtOffComeIn", (int)workAtOfficeComeIn.minutesSinceMidnight);
  pref.putInt("workAtOffSleep", (int)workAtOfficeSleep.minutesSinceMidnight);
  pref.putInt("weekendWakeUp", (int)weekendWakeUp.minutesSinceMidnight);
  pref.putInt("weekendSleep", (int)weekendSleep.minutesSinceMidnight);
  pref.putInt("regularWeek0", (int)regularWeek[0]);
  pref.putInt("regularWeek1", (int)regularWeek[1]);
  pref.putInt("regularWeek2", (int)regularWeek[2]);
  pref.putInt("regularWeek3", (int)regularWeek[3]);
  pref.putInt("regularWeek4", (int)regularWeek[4]);
  pref.putInt("regularWeek5", (int)regularWeek[5]);
  pref.putInt("regularWeek6", (int)regularWeek[6]);
  pref.putFloat("highTemp", highTemp);
  pref.putFloat("lowTemp", lowTemp);
  pref.putBool("overrideToday", overrideToday);
  pref.putBool("overrideTomrrw", overrideTomorrow);
  pref.putBool("overrideMltpl", overrideMultiple);
  pref.putInt("overrideMultCnt", (int)overrideMultipleCount);
  pref.putBool("multipleForever", multipleForever);
  pref.putFloat("sensorOffset", sensorOffset);
  pref.end();
  settingsChanged = false;
  return true;
}

bool ControllerData_t::loadSettingsNVS() {
  Preferences pref;
  if (!pref.begin("controller", true)) {   // true = read-only
    Serial.println("Failed to open NVS for reading");
    return false;
  }
}

  workFromHomeWakeUp.minutesSinceMidnight = pref.getInt("workFromHmeWkUp", 7*60+ 0);
  workFromHomeSleep.minutesSinceMidnight = pref.getInt("workFromHmeSlep", 21*60+30);
  workAtOfficeWakeUp.minutesSinceMidnight = pref.getInt("workAtOffWakeUp", 6*60+30);
  workAtOfficeGoOut.minutesSinceMidnight = pref.getInt("workAtOffGoOut", 7*60+15);
  workAtOfficeComeIn.minutesSinceMidnight = pref.getInt("workAtOffComeIn", 17*60+15);
  workAtOfficeSleep.minutesSinceMidnight = pref.getInt("workAtOffSleep", 21*60+30);
  weekendWakeUp.minutesSinceMidnight = pref.getInt("weekendWakeUp", 9*60+30);
  weekendSleep.minutesSinceMidnight = pref.getInt("weekendSleep", 22*60+ 0);
  regularWeek[0] = pref.getInt("regularWeek0", 2);
  regularWeek[1] = pref.getInt("regularWeek1", 1);
  regularWeek[2] = pref.getInt("regularWeek2", 0);
  regularWeek[3] = pref.getInt("regularWeek3", 1);
  regularWeek[4] = pref.getInt("regularWeek4", 1);
  regularWeek[5] = pref.getInt("regularWeek5", 0);
  regularWeek[6] = pref.getInt("regularWeek6", 2);
  highTemp = pref.getFloat("highTemp", 19.5);
  lowTemp = pref.getFloat("lowTemp", 17);
  overrideToday = pref.getBool("overrideToday", false);
  overrideTomorrow = pref.getBool("overrideTomrrw", false);
  overrideMultiple = pref.getBool("overrideMltpl", false);
  overrideMultipleCount = pref.getInt("overrideMultCnt", 0);
  multipleForever = pref.getBool("multipleForever", false);
  sensorOffset = pref.getFloat("sensorOffset", -3.5);
  pref.end();
  return true;
}
```

In [ ]:
STR="""scHomeScreen, scMain, scOverruleToday, scOverruleTomorrow, scnOverruleMultiple, scnOverruleMultipleSchedule, scnSettingsMain, scnSettingsTemperature, scnSettingsWeekSchedule,
  scnSettingsDaySchedule, scnSettingsHomeTimes, scnSettingsOfficeTimes, scnSettingsWeekendTimes, scnSettingsSensorOffset"""

commands = STR.split(",")
commands = [cmd.replace("\n","").replace(" ","") for cmd in commands]
for cmd in commands:
    print(cmd)

import pyperclip
pyperclip.copy('\n'.join(commands))

scHomeScreen
scMain
scOverruleToday
scOverruleTomorrow
scnOverruleMultiple
scnOverruleMultipleSchedule
scnSettingsMain
scnSettingsTemperature
scnSettingsWeekSchedule
scnSettingsDaySchedule
scnSettingsHomeTimes
scnSettingsOfficeTimes
scnSettingsWeekendTimes
scnSettingsSensorOffset


In [ ]:
```
#include <Preferences.h>

bool ControllerData_t::saveSettingsNVS() {
  Preferences pref;
  if (!pref.begin("controller", false)) {
    Serial.println("Failed to open NVS for writing");
    return false;
  }

  pref.putInt("workFromHmeWkUp", (int)workFromHomeWakeUp.minutesSinceMidnight);
  pref.putInt("workFromHmeSlep", (int)workFromHomeSleep.minutesSinceMidnight);
  pref.putInt("workAtOffWakeUp", (int)workAtOfficeWakeUp.minutesSinceMidnight);
  pref.putInt("workAtOffGoOut", (int)workAtOfficeGoOut.minutesSinceMidnight);
  pref.putInt("workAtOffComeIn", (int)workAtOfficeComeIn.minutesSinceMidnight);
  pref.putInt("workAtOffSleep", (int)workAtOfficeSleep.minutesSinceMidnight);
  pref.putInt("weekendWakeUp", (int)weekendWakeUp.minutesSinceMidnight);
  pref.putInt("weekendSleep", (int)weekendSleep.minutesSinceMidnight);
  pref.putInt("regularWeek0", (int)regularWeek[0]);
  pref.putInt("regularWeek1", (int)regularWeek[1]);
  pref.putInt("regularWeek2", (int)regularWeek[2]);
  pref.putInt("regularWeek3", (int)regularWeek[3]);
  pref.putInt("regularWeek4", (int)regularWeek[4]);
  pref.putInt("regularWeek5", (int)regularWeek[5]);
  pref.putInt("regularWeek6", (int)regularWeek[6]);
  pref.putFloat("highTemp", highTemp);
  pref.putFloat("lowTemp", lowTemp);
  pref.putBool("overrideToday", overrideToday);
  pref.putBool("overrideTomrrw", overrideTomorrow);
  pref.putBool("overrideMltpl", overrideMultiple);
  pref.putInt("overrideMultCnt", (int)overrideMultipleCount);
  pref.putBool("multipleForever", multipleForever);
  pref.putFloat("sensorOffset", sensorOffset);
  pref.end();
  settingsChanged = false;
  return true;
}

bool ControllerData_t::loadSettingsNVS() {
  Preferences pref;
  if (!pref.begin("controller", true)) {   // true = read-only
    Serial.println("Failed to open NVS for reading");
    return false;
  }
}

  workFromHomeWakeUp.minutesSinceMidnight = pref.getInt("workFromHmeWkUp", 7*60+ 0);
  workFromHomeSleep.minutesSinceMidnight = pref.getInt("workFromHmeSlep", 21*60+30);
  workAtOfficeWakeUp.minutesSinceMidnight = pref.getInt("workAtOffWakeUp", 6*60+30);
  workAtOfficeGoOut.minutesSinceMidnight = pref.getInt("workAtOffGoOut", 7*60+15);
  workAtOfficeComeIn.minutesSinceMidnight = pref.getInt("workAtOffComeIn", 17*60+15);
  workAtOfficeSleep.minutesSinceMidnight = pref.getInt("workAtOffSleep", 21*60+30);
  weekendWakeUp.minutesSinceMidnight = pref.getInt("weekendWakeUp", 9*60+30);
  weekendSleep.minutesSinceMidnight = pref.getInt("weekendSleep", 22*60+ 0);
  regularWeek[0] = pref.getInt("regularWeek0", 2);
  regularWeek[1] = pref.getInt("regularWeek1", 1);
  regularWeek[2] = pref.getInt("regularWeek2", 0);
  regularWeek[3] = pref.getInt("regularWeek3", 1);
  regularWeek[4] = pref.getInt("regularWeek4", 1);
  regularWeek[5] = pref.getInt("regularWeek5", 0);
  regularWeek[6] = pref.getInt("regularWeek6", 2);
  highTemp = pref.getFloat("highTemp", 19.5);
  lowTemp = pref.getFloat("lowTemp", 17);
  overrideToday = pref.getBool("overrideToday", false);
  overrideTomorrow = pref.getBool("overrideTomrrw", false);
  overrideMultiple = pref.getBool("overrideMltpl", false);
  overrideMultipleCount = pref.getInt("overrideMultCnt", 0);
  multipleForever = pref.getBool("multipleForever", false);
  sensorOffset = pref.getFloat("sensorOffset", -3.5);
  pref.end();
  return true;
}
```

SyntaxError: unmatched '}' (2617792930.py, line 72)

In [ ]:
SCN = """  { scnHome,                     { cmdSetpointHigher, cmdMenuMain, cmdSetpointLower } },
  { scnMain,                     { cmdMenuOverruleToday, cmdMenuOverruleTomorrow, cmdMenuOverruleMultipleDays, cmdMenuSettings, cmdMenuHome} },
  { scnOverruleToday,            { cmdOverruleTodayWorkFromHome, cmdOverruleTodayWorkAtOffice, cmdOverruleTodayWeekend, cmdOverruleTodayAway, cmdOverruleTodayAutomatic, cmdMenuMain } },
  { scnOverruleTomorrow,         { cmdOverruleTomorrowWorkFromHome, cmdOverruleTomorrowWorkAtOffice, cmdOverruleTomorrowWeekend, cmdOverruleTomorrowAway, cmdOverruleTomorrowAutomatic, cmdMenuMain } },
  { scnOverruleMultiple,         { cmdOverruleMultipleMoreDays, cmdOverruleMultipleFewerDays, cmdMultipleDaySchedule, cmdOverruleMultipleAutomatic, cmdMenuMain } },
  { scnOverruleMultipleSchedule, { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdOverruleTodayAway, cmdOverruleTodayAutomatic, cmdMenuOverruleMultipleDays } },
  { scnSettingsMain,             { cmdMenuTemperature, cmdMenuSensorOffset, cmdMenuWeekSchedule, cmdMenuHomeTimes, cmdMenuOfficeTimes, cmdMenuWeekendTimes, cmdMenuMain } },
  { scnSettingsTemperature,      { cmdHighTemperatureUp, cmdHighTemperatureDown, cmdLowTemperatureUp, cmdLowTemperatureDown, cmdMenuSettings } },
  { scnSettingsSensorOffset,     { cmdSensorOffsetUp, cmdSensorOffsetDown, cmdMenuSettings } },
  { scnSettingsWeekSchedule,     { cmdMonday, cmdTuesday, cmdWednesday, cmdThursday, cmdFriday, cmdSaturday, cmdSunday, cmdMenuSettings} },
  { scnSettingsDaySchedule,      { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdOverruleTodayAway, cmdMenuWeekSchedule } },
  { scnSettingsHomeTimes,        { cmdHomeWakeUpLater, cmdHomeWakeUpEarlier, cmdHomeGoToSleepLater, cmdHomeGoToSleepEarlier, cmdMenuSettings } },
  { scnSettingsOfficeTimes,      { cmdOfficeWakeUpLater, cmdOfficeWakeUpEarlier, cmdOfficeLeaveLater, cmdOfficeLeaveEarlier, cmdOfficeComeHomeLater, cmdOfficeComeHomeEarlier, cmdHomeGoToSleepLater, cmdHomeGoToSleepEarlier, cmdMenuSettings } },
  { scnSettingsWeekendTimes,     { cmdWeekendWakeUpLater, cmdWeekendWakeUpEarlier, cmdWeekendGoToSleepLater, cmdWeekendGoToSleepEarlier, cmdMenuSettings } }"""

lines=SCN.split("\n")

result=""
for line in lines:
    sections = line.replace("}", "").replace("{", "").split(",")
    screen=sections[0].strip()
    try:
      commands=[cmd.strip() for cmd in sections[1:] ]
    except:
      print("error: ", sections)
      print(line.replace("}", "").replace("{", ""))
      break

    result+=screen+"\n"
    for cmd in commands:
        #result+=screen+"\t"+title+"\t"+cmd+"\n"
        result+=cmd+"\n"

print(result)
pyperclip.copy(result)

scnHome
cmdSetpointHigher
cmdMenuMain
cmdSetpointLower

scnMain
cmdMenuOverruleToday
cmdMenuOverruleTomorrow
cmdMenuOverruleMultipleDays
cmdMenuSettings
cmdMenuHome

scnOverruleToday
cmdOverruleTodayWorkFromHome
cmdOverruleTodayWorkAtOffice
cmdOverruleTodayWeekend
cmdOverruleTodayAway
cmdOverruleTodayAutomatic
cmdMenuMain

scnOverruleTomorrow
cmdOverruleTomorrowWorkFromHome
cmdOverruleTomorrowWorkAtOffice
cmdOverruleTomorrowWeekend
cmdOverruleTomorrowAway
cmdOverruleTomorrowAutomatic
cmdMenuMain

scnOverruleMultiple
cmdOverruleMultipleMoreDays
cmdOverruleMultipleFewerDays
cmdMultipleDaySchedule
cmdOverruleMultipleAutomatic
cmdMenuMain

scnOverruleMultipleSchedule
cmdWorkFromHome
cmdWorkAtOffice
cmdWeekend
cmdOverruleTodayAway
cmdOverruleTodayAutomatic
cmdMenuOverruleMultipleDays

scnSettingsMain
cmdMenuTemperature
cmdMenuSensorOffset
cmdMenuWeekSchedule
cmdMenuHomeTimes
cmdMenuOfficeTimes
cmdMenuWeekendTimes
cmdMenuMain

scnSettingsTemperature
cmdHighTemperatureUp
cmdHighTemperatureDow

In [ ]:
TLGRM = """  { cmdSetpointLower,                { "xxx°C",                  "/cmdSetpointLower" } },
  { cmdSetpointAuto,                 { "Auto",                   "/cmdSetpointAuto" } },
  { cmdSetpointHigher,               { "xxx°C",                  "/cmdSetpointHigher" } },
  { cmdGoAway,                       { "Go out",                 "/cmdGoAway" } },
  { cmdMenuOverruleToday,            { "Overrule today",         "/cmdMenuOverruleToday" } },
  { cmdMenuOverruleTomorrow,         { "Overrule tomorrow",      "/cmdMenuOverruleTomorrow" } },
  { cmdMenuOverruleMultipleDays,     { "Overrule multiple days", "/cmdMenuOverruleMultipleDays" } },
  { cmdMenuSettings,                 { "Settings",               "/cmdMenuSettings" } },
  { cmdUpdateStatus,                 { "Status",                 "/cmdUpdateStatus" } },
  { cmdOverruleTodayWorkFromHome,    { "Work from home",         "/cmdOverruleTodayWorkFromHome" } },
  { cmdOverruleTodayWorkAtOffice,    { "Work from office",       "/cmdOverruleTodayWorkAtOffice" } },
  { cmdOverruleTodayWeekend,         { "Weekend",                "/cmdOverruleTodayWeekend" } },
  { cmdOverruleTodayAway,            { "All day away",           "/cmdOverruleTodayAway" } },
  { cmdOverruleTodayAutomatic,       { "Automatic",              "/cmdOverruleTodayAutomatic"       } },
  { cmdMenuMain,                     { "Back to main menu",      "/cmdMenuMain"                     } },

  { cmdOverruleTomorrowWorkFromHome, { "Work from home",         "/cmdOverruleTomorrowWorkFromHome" } },
  { cmdOverruleTomorrowWorkAtOffice, { "Work from office",       "/cmdOverruleTomorrowWorkAtOffice" } },
  { cmdOverruleTomorrowWeekend,      { "Weekend",                "/cmdOverruleTomorrowWeekend"      } },
  { cmdOverruleTomorrowAway,         { "All day away",           "/cmdOverruleTomorrowAway"         } },
  { cmdOverruleTomorrowAutomatic,    { "Automatic",              "/cmdOverruleTomorrowAutomatic"    } },

  { cmdOverruleMultipleWorkFromHome, { "Work from home",         "/cmdOverruleMultipleWorkFromHome" } },
  { cmdOverruleMultipleWorkAtOffice, { "Work from office",       "/cmdOverruleMultipleWorkAtOffice" } },
  { cmdOverruleMultipleWeekend,      { "Weekend",                "/cmdOverruleMultipleWeekend"      } },
  { cmdOverruleMultipleAway,         { "All day away",           "/cmdOverruleMultipleAway" } },
  { cmdOverruleMultipleAutomatic,    { "Automatic",              "/cmdOverruleMultipleAutomatic" } },
  { cmdOverruleMultipleFewerDays,    { "..days",                 "/cmdOverruleMultipleFewerDays" } },
  { cmdOverruleMultipleMoreDays,     { "..days",                 "/cmdOverruleMultipleMoreDays"   } },

  { cmdMenuWeekSchedule,             { "Weekly schedule",        "/cmdMenuWeekSchedule"          } },
  { cmdMenuHomeTimes,                { "Home times",             "/cmdMenuHomeTimes"           } },
  { cmdMenuOfficeTimes,              { "Office times",           "/cmdMenuOfficeTimes"         } },
  { cmdMenuWeekendTimes,             { "Weekend times",          "/cmdMenuWeekendTimes"        } },
  { cmdMenuTemperature,              { "Temperatures",           "/cmdMenuTemperature"         } },
  { cmdMenuSensorOffset,             { "Sensor offset",          "/cmdMenuSensorOffset"        } },
  { cmdReportBoiler,                 { "Logdata",                "/cmdReportBoiler"            } },
  { cmdReportTiming,                 { "Timing report",          "/cmdReportTiming"            } },

  { cmdMonday,                       { "Monday",                 "/cmdMonday"                  } },
  { cmdTuesday,                      { "Tuesday",                "/cmdTuesday"                 } },
  { cmdWednesday,                    { "Wednesday",              "/cmdWednesday"               } },
  { cmdThursday,                     { "Thursday",               "/cmdThursday"                } },
  { cmdFriday,                       { "Friday",                 "/cmdFriday"                  } },
  { cmdSaturday,                     { "Saturday",               "/cmdSaturday"                } },
  { cmdSunday,                       { "Sunday",                 "/cmdSunday"                  } },

  { cmdWorkFromHome,                 { "Work from home",         "/cmdWorkFromHome"            } },
  { cmdWorkAtOffice,                 { "Work from office",       "/cmdWorkAtOffice"            } },
  { cmdWeekend,                      { "Weekend day",            "/cmdWeekend"                 } },
  { cmdAllDayAway,                   { "All day away",           "/cmdAllDayAway"              } },

  { cmdHomeWakeUpEarlier,            { "A: Wake up -",           "/cmdHomeWakeUpEarlier"       } },
  { cmdHomeWakeUpLater,              { "A: Wake up +",           "/cmdHomeWakeUpLater"         } },
  { cmdHomeGoToSleepEarlier,         { "B: Sleep -",             "/cmdHomeGoToSleepEarlier"    } },
  { cmdHomeGoToSleepLater,           { "B: Sleep +",             "/cmdHomeGoToSleepLater"      } },

  { cmdOfficeWakeUpEarlier,          { "A: Wake up -",           "/cmdOfficeWakeUpEarlier"     } },
  { cmdOfficeWakeUpLater,            { "A: Wake up +",           "/cmdOfficeWakeUpLater"       } },
  { cmdOfficeLeaveEarlier,           { "B: Go out -",            "/cmdOfficeLeaveEarlier"      } },
  { cmdOfficeLeaveLater,             { "B: Go out +",            "/cmdOfficeLeaveLater"        } },
  { cmdOfficeComeHomeEarlier,        { "C: Come in -",           "/cmdOfficeComeHomeEarlier"   } },
  { cmdOfficeComeHomeLater,          { "C: Come in +",           "/cmdOfficeComeHomeLater"     } },
  { cmdOfficeGoToSleepEarlier,       { "D: Sleep -",             "/cmdOfficeGoToSleepEarlier"  } },
  { cmdOfficeGoToSleepLater,         { "D: Sleep +",             "/cmdOfficeGoToSleepLater"    } },

  { cmdWeekendWakeUpEarlier,         { "A: Wake up -",           "/cmdWeekendWakeUpEarlier"    } },
  { cmdWeekendWakeUpLater,           { "A: Wake up +",           "/cmdWeekendWakeUpLater"      } },
  { cmdWeekendGoToSleepEarlier,      { "B: Sleep -",             "/cmdWeekendGoToSleepEarlier" } },
  { cmdWeekendGoToSleepLater,        { "B: Sleep +",             "/cmdWeekendGoToSleepLater"   } },

  { cmdHighTemperatureDown,          { "High temp ..°C",         "/cmdHighTemperatureDown"     } },
  { cmdHighTemperatureUp,            { "High temp ..°C",         "/cmdHighTemperatureUp"       } },
  { cmdLowTemperatureDown,           { "Low temp ..°C",          "/cmdLowTemperatureDown"      } },
  { cmdLowTemperatureUp,             { "Low temp ..°C",          "/cmdLowTemperatureUp"        } },

  { cmdSensorOffsetDown,             { "Offset down",            "/cmdSensorOffsetDown"        } },
  { cmdSensorOffsetUp,               { "Offset up",              "/cmdSensorOffsetUp"          } }
"""

lines = TLGRM.split('\n')
commands=[]
mxCmd = 0
mxClb = 0
mxTtl = 0
for line in lines:
    sections = line.replace("{","").replace("}","").split(',')
    if(len(sections)>1):
        cmd = sections[0].replace(" ","").replace("tc","")
        title= sections[1].replace('"','').strip()
        callback= sections[2].replace('"','')

        if mxTtl<len(title):
          mxTtl=len(title)
        if mxCmd<len(cmd):
          mxCmd=len(cmd)
        if mxClb<len(callback):
          mxClb=len(callback)

        commands.append( ( cmd, title, callback) )

results=''
for cmd, title, callback in commands:
  results+='{ '+cmd + ', '+ ' '*(mxCmd-len(cmd)) + '{ "' + title + '", ' + '"'+ callback + '" } },'

pyperclip.copy(results)
print(results)


{ cmdSetpointLower,                { "xxx°C", "                  /cmdSetpointLower  " } },{ cmdSetpointAuto,                 { "Auto", "                   /cmdSetpointAuto  " } },{ cmdSetpointHigher,               { "xxx°C", "                  /cmdSetpointHigher  " } },{ cmdGoAway,                       { "Go out", "                 /cmdGoAway  " } },{ cmdMenuOverruleToday,            { "Overrule today", "         /cmdMenuOverruleToday  " } },{ cmdMenuOverruleTomorrow,         { "Overrule tomorrow", "      /cmdMenuOverruleTomorrow  " } },{ cmdMenuOverruleMultipleDays,     { "Overrule multiple days", " /cmdMenuOverruleMultipleDays  " } },{ cmdMenuSettings,                 { "Settings", "               /cmdMenuSettings  " } },{ cmdUpdateStatus,                 { "Status", "                 /cmdUpdateStatus  " } },{ cmdOverruleTodayWorkFromHome,    { "Work from home", "         /cmdOverruleTodayWorkFromHome  " } },{ cmdOverruleTodayWorkAtOffice,    { "Work from office", "       /cmdOverru

In [ ]:
NEW_CMD = """cmdMenuMain
cmdSetpointHigher
cmdSetpointLower
cmdSetpointAuto
cmdGoAway
cmdUpdateStatus

cmdMenuOverruleToday
cmdMenuOverruleTomorrow
cmdMenuOverruleMultipleDays
cmdMenuSettings
cmdMenuHome

cmdTodayWorkFromHome
cmdTodayWorkAtOffice
cmdTodayWeekend
cmdTodayAllDayAway
cmdTodayAutomatic
cmdMenuMain

cmdTomorrowWorkFromHome
cmdTomorrowWorkAtOffice
cmdTomorrowWeekend
cmdTomorrowAllDayAway
cmdTomorrowAutomatic
cmdMenuMain

cmdMultipleMoreDays
cmdMultipleFewerDays
cmdMultipleDaySchedule
cmdMultipleAutomatic
cmdMenuMain

cmdMultipleWorkFromHome
cmdMultipleWorkAtOffice
cmdMultipleWeekend
cmdMultipleAllDayAway
cmdMultipleAutomatic


cmdMenuWeekSchedule
cmdMenuHomeTimes
cmdMenuOfficeTimes
cmdMenuWeekendTimes
cmdMenuTemperature
cmdMenuSensorOffset
cmdReportBoiler
cmdReportTiming
cmdMenuMain


cmdMonday
cmdTuesday
cmdWednesday
cmdThursday
cmdFriday
cmdSaturday
cmdSunday
cmdMenuSettings

cmdWorkFromHome
cmdWorkAtOffice
cmdWeekend
cmdAllDayAway
cmdMenuWeekSchedule

cmdHomeWakeUpLater
cmdHomeWakeUpEarlier
cmdHomeGoToSleepLater
cmdHomeGoToSleepEarlier
cmdMenuSettings

cmdOfficeWakeUpLater
cmdOfficeWakeUpEarlier
cmdOfficeLeaveLater
cmdOfficeLeaveEarlier
cmdOfficeComeHomeLater
cmdOfficeComeHomeEarlier
cmdOfficeGoToSleepLater
cmdOfficeGoToSleepEarlier
cmdMenuSettings

cmdWeekendWakeUpLater
cmdWeekendWakeUpEarlier
cmdWeekendGoToSleepLater
cmdWeekendGoToSleepEarlier
cmdMenuSettings

cmdHighTemperatureUp
cmdHighTemperatureDown
cmdLowTemperatureUp
cmdLowTemperatureDown
cmdMenuSettings

cmdSensorOffsetUp
cmdSensorOffsetDown
cmdMenuSettings

cmdCommandNotRecognized
"""

# Keep order but remove duplicates
lines = NEW_CMD.split("\n")
commands = []
for line in lines:
    cmd=line.strip()
    if len(cmd)>0 and cmd not in commands:
        commands.append(cmd)

LABELS = """cmdMenuMain	Main menu
cmdSetpointHigher	Setpoint higher
cmdSetpointLower	Setpoint lower
cmdSetpointAuto	Setpoint automatic
cmdGoAway	Go out
cmdUpdateStatus	Update

cmdMenuOverruleToday	Overrule today
cmdMenuOverruleTomorrow	Overrule tomorrow
cmdMenuOverruleMultipleDays	Overrule multiple days
cmdMenuSettings	Settings
cmdMenuHome	Home menu

cmdTodayWorkFromHome	Work from home
cmdTodayWorkAtOffice	Work at office
cmdTodayWeekend	Weekend
cmdTodayAllDayAway	All day away
cmdTodayAutomatic	Automatic
cmdMenuMain	Main menu

cmdTomorrowWorkFromHome	Work from home
cmdTomorrowWorkAtOffice	Work from office
cmdTomorrowWeekend	Weekend
cmdTomorrowAllDayAway	All day away
cmdTomorrowAutomatic	Automatic
cmdMenuMain	Main menu

cmdMultipleMoreDays	More days
cmdMultipleFewerDays	Fewer days
cmdMultipleDaySchedule	Set day schedule
cmdMultipleAutomatic	Switch off multiple
cmdMenuMain	Main menu

cmdMultipleWorkFromHome	Work from home
cmdMultipleWorkAtOffice	Work from office
cmdMultipleWeekend	Weekend
cmdMultipleAllDayAway	All day away
cmdMultipleAutomatic	Automatic


cmdMenuWeekSchedule	Set weekly schedule
cmdMenuHomeTimes	Set home times
cmdMenuOfficeTimes	Set office times
cmdMenuWeekendTimes	Set weekend times
cmdMenuTemperature	Set temperatures
cmdMenuSensorOffset	Adjust sensor offset
cmdReportBoiler	Report boiler
cmdReportTiming	Report timing
cmdMenuMain


cmdMonday	Monday
cmdTuesday	Tuesday
cmdWednesday	Wednesday
cmdThursday	Thursday
cmdFriday	Friday
cmdSaturday	Saturday
cmdSunday	Sunday
cmdMenuSettings

cmdWorkFromHome	Work from home
cmdWorkAtOffice	Work from office
cmdWeekend	Weekend day
cmdAllDayAway	All day away
cmdMenuWeekSchedule

cmdHomeWakeUpLater	Wake up later
cmdHomeWakeUpEarlier	Wake up earlier
cmdHomeGoToSleepLater	Go to sleep later
cmdHomeGoToSleepEarlier	Go to sleep earlier
cmdMenuSettings

cmdOfficeWakeUpLater	Wake up later
cmdOfficeWakeUpEarlier	Wake up earlier
cmdOfficeLeaveLater	Leave later
cmdOfficeLeaveEarlier	Leave earlier
cmdOfficeComeHomeLater	Come in later
cmdOfficeComeHomeEarlier	Come in earlier
cmdOfficeGoToSleepLater	Go to sleep later
cmdOfficeGoToSleepEarlier	Go to sleep earlier
cmdMenuSettings

cmdWeekendWakeUpLater	Wake up later
cmdWeekendWakeUpEarlier	Wake up earlier
cmdWeekendGoToSleepLater	Go to sleep later
cmdWeekendGoToSleepEarlier	Go to sleep earlier
cmdMenuSettings

cmdHighTemperatureUp	High temp up
cmdHighTemperatureDown	High temp down
cmdLowTemperatureUp	Low temp up
cmdLowTemperatureDown	Low temp down
cmdMenuSettings

cmdSensorOffsetUp	Offset up
cmdSensorOffsetDown	Offset down
cmdMenuSettings	"""

labels={}
for line in LABELS.split('\n'):
    if len(line)>0:
        command, label = line.split('\t')
        if len(label)>0:
            labels[command]=label

import pyperclip
results=''
for command in commands:
    try:
        results+='{' + command +', "'+ labels[command] + '" },\n'
    except:
        print(command+' not found')

# Add text for responseMessage
results=''
for command in commands:
    try:
        results+='  // case ' + command +': response=""; break;\n'
    except:
        print(command+' not found')

pyperclip.copy(results)


cmdCommandNotRecognized not found


In [ ]:
TGM_MENU = """scnMain, scnOverruleToday, scnOverruleTomorrow, scnOverruleMultiple, scnSettingsMain, scnSettingsSensorOffset, scnSettingsWeekSchedule,
  scnSettingsDaySchedule, scnSettingsHomeTimes, scnSettingsOfficeTimes, scnSettingsWeekendTimes, scnSettingsTemperature"""

menus= [ mn.replace(' ','').replace('\n','') for mn in TGM_MENU.split(',')]

pyperclip.copy('\n'.join(menus))


In [ ]:
CONTROLLER_CMDS = """    void setTemperatureOffset()
    void increaseSensorOffset()
    void decreaseSensorOffset()
    void setpointLower();
    void setpointAuto();
    void setpointHigher();
    void goAway();
    void overruleTodayWorkFromHome();
    void overruleTodayWorkFromOffice();
    void overruleTodayWeekend();
    void overruleTodayAway();
    void overruleTodayAutomatic();
    void overruleTomorrowWorkFromHome();
    void overruleTomorrowWorkFromOffice();
    void overruleTomorrowWeekend();
    void overruleTomorrowAway();
    void overruleTomorrowAutomatic();
    void overruleMultipleWorkFromHome();
    void overruleMultipleWorkFromOffice();
    void overruleMultipleWeekend();
    void overruleMultipleAway();
    void overruleMultipleAutomatic();
    void overruleMultipleLessDays();
    void overruleMultipleMoreDays();
    void setWeekSchedule
    void workFromHomeWakeUpEarlier();
    void workFromHomeWakeUpLater();
    void workFromHomeSleepEarlier();
    void workFromHomeSleepLater();
    void workFromOfficeWakeUpEarlier();
    void workFromOfficeWakeUpLater();
    void workFromOfficeGoOutEarlier();
    void workFromOfficeGoOutLater();
    void workFromOfficeComeInEarlier();
    void workFromOfficeComeInLater();
    void workFromOfficeSleepEarlier();
    void workFromOfficeSleepLater();
    void weekendWakeUpEarlier();
    void weekendWakeUpLater();
    void weekendSleepEarlier();
    void weekendSleepLater();
    void highTempLower();
    void highTempHigher();
    void lowTempLower();
    void lowTempHigher();
"""

import pyperclip
results=[]
makeif = []
for cmd in [l.replace('void', '').replace('()','').replace(';','').strip() for l in CONTROLLER_CMDS.split('\n')]:
    if(len(cmd)>0):
        ctr= 'ctr' + cmd[0].upper() + cmd[1:]
        results.append(ctr)
        makeif.append('else if( messageToController.command == ' + ctr + ' ) ' + cmd + '();')
#pyperclip.copy(', '.join(results))
pyperclip.copy('\n'.join(makeif))


In [ ]:
SCREENS = """{ scnHome,                     {"Home" ,                        { cmdSetpointHigher, cmdMenuMain, cmdSetpointLower } } },
    { scnMain,                     {"Main menu",                    { cmdMenuOverruleToday, cmdMenuOverruleTomorrow, cmdMenuOverruleMultipleDays, cmdMenuSettings, cmdMenuHome} } },
    { scnOverruleToday,            {"Overrule today",               { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdTodayAllDayAway, cmdTodayAutomatic, cmdMenuMain } } },
    { scnOverruleTomorrow,         {"Overrule tomorrow",            { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdTodayAllDayAway, cmdTodayAutomatic, cmdMenuMain } } },
    { scnOverruleMultiple,         {"Overrule multiple days",       { cmdMultipleMoreDays, cmdMultipleFewerDays, cmdMultipleDaySchedule, cmdMultipleAutomatic, cmdMenuMain } } },
    { scnOverruleMultipleSchedule, {"Overrule multiple - schedule", { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdTodayAllDayAway, cmdTodayAutomatic, cmdMenuOverruleMultipleDays } } },
    { scnSettingsMain,             {"Settings",                     { cmdMenuTemperature, cmdMenuSensorOffset, cmdMenuWeekSchedule, cmdMenuHomeTimes, cmdMenuOfficeTimes, cmdMenuWeekendTimes, cmdMenuMain } } },
    { scnSettingsTemperature,      {"Temperature settings",         { cmdHighTemperatureUp, cmdHighTemperatureDown, cmdLowTemperatureUp, cmdLowTemperatureDown, cmdMenuSettings } } },
    { scnSettingsSensorOffset,     {"Adjust sensor offset",         { cmdSensorOffsetUp, cmdSensorOffsetDown, cmdMenuSettings } } },
    { scnSettingsWeekSchedule,     {"Set week schedule",            { cmdMonday, cmdTuesday, cmdWednesday, cmdThursday, cmdFriday, cmdSaturday, cmdSunday, cmdMenuSettings} } },
    { scnSettingsDaySchedule,      {"Set day schedule",             { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdTodayAllDayAway, cmdMenuWeekSchedule } } },
    { scnSettingsHomeTimes,        {"Set home times",               { cmdHomeWakeUpLater, cmdHomeWakeUpEarlier, cmdHomeGoToSleepLater, cmdHomeGoToSleepEarlier, cmdMenuSettings } } },
    { scnSettingsOfficeTimes,      {"Set office times",             { cmdOfficeWakeUpLater, cmdOfficeWakeUpEarlier, cmdOfficeLeaveLater, cmdOfficeLeaveEarlier, cmdOfficeComeHomeLater, cmdOfficeComeHomeEarlier, cmdHomeGoToSleepLater, cmdHomeGoToSleepEarlier, cmdMenuSettings } } },
    { scnSettingsWeekendTimes,     {"Set weekend times",            { cmdWeekendWakeUpLater, cmdWeekendWakeUpEarlier, cmdWeekendGoToSleepLater, cmdWeekendGoToSleepEarlier, cmdMenuSettings } } }"""

results=[]
for line in SCREENS.split('\n'):
    elements = line.split('{')
    screenTag = elements[1].replace(',','').strip()
    screenTitle = elements[2].replace(',','').replace('"','').strip()

    results.append('  case ' + screenTag + ": showSubMenuLines = 2; subMenuLine1 = ''; subMenuLine2 = ''; break;" )
    #results.append(screenTag + '\t' + screenTitle)
    menuItems= elements[3].split(',')
    for menuItem in menuItems:
        #results.append('\t\t'+menuItem.replace('}','').strip())
        pass

import pyperclip
pyperclip.copy('\n'.join(results))

In [ ]:
CTR = """ctrControl, ctrSetTemperatureOffset, ctrIncreaseSensorOffset, ctrDecreaseSensorOffset,
  ctrSetpointLower, ctrSetpointAuto, ctrSetpointHigher, ctrGoAway, ctrOverruleTodayWorkFromHome,
  ctrOverruleTodayWorkFromOffice, ctrOverruleTodayWeekend, ctrOverruleTodayAway, ctrOverruleTodayAutomatic,
  ctrOverruleTomorrowWorkFromHome, ctrOverruleTomorrowWorkFromOffice, ctrOverruleTomorrowWeekend,
  ctrOverruleTomorrowAway, ctrOverruleTomorrowAutomatic, ctrOverruleMultipleWorkFromHome,
  ctrOverruleMultipleWorkFromOffice, ctrOverruleMultipleWeekend, ctrOverruleMultipleAway,
  ctrOverruleMultipleAutomatic, ctrOverruleMultipleLessDays, ctrOverruleMultipleMoreDays,
  ctrSetWeekSchedule, ctrWorkFromHomeWakeUpEarlier, ctrWorkFromHomeWakeUpLater, ctrWorkFromHomeSleepEarlier,
  ctrWorkFromHomeSleepLater, ctrWorkFromOfficeWakeUpEarlier, ctrWorkFromOfficeWakeUpLater,
  ctrWorkFromOfficeGoOutEarlier, ctrWorkFromOfficeGoOutLater, ctrWorkFromOfficeComeInEarlier,
  ctrWorkFromOfficeComeInLater, ctrWorkFromOfficeSleepEarlier, ctrWorkFromOfficeSleepLater,
  ctrWeekendWakeUpEarlier, ctrWeekendWakeUpLater, ctrWeekendSleepEarlier, ctrWeekendSleepLater,
  ctrHighTempLower, ctrHighTempHigher, ctrLowTempLower, ctrLowTempHigher, ctrDoNothing"""

MNU = """cmdMenuMain, cmdSetpointHigher, cmdSetpointLower, cmdSetpointAuto, cmdGoAway,
cmdUpdateStatus, cmdMenuOverruleToday, cmdMenuOverruleTomorrow, cmdMenuOverruleMultipleDays,
cmdMenuSettings, cmdMenuHome, cmdTodayWorkFromHome, cmdTodayWorkAtOffice, cmdTodayWeekend,
cmdTodayAllDayAway, cmdTodayAutomatic, cmdTomorrowWorkFromHome, cmdTomorrowWorkAtOffice,
cmdTomorrowWeekend, cmdTomorrowAllDayAway, cmdTomorrowAutomatic, cmdMultipleMoreDays,
cmdMultipleFewerDays, cmdMultipleDaySchedule, cmdMultipleAutomatic, cmdMultipleWorkFromHome,
cmdMultipleWorkAtOffice, cmdMultipleWeekend, cmdMultipleAllDayAway, cmdMenuWeekSchedule,
cmdMenuHomeTimes, cmdMenuOfficeTimes, cmdMenuWeekendTimes, cmdMenuTemperature, cmdMenuSensorOffset,
cmdReportBoiler, cmdReportTiming, cmdMonday, cmdTuesday, cmdWednesday, cmdThursday, cmdFriday,
cmdSaturday, cmdSunday, cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdAllDayAway, cmdHomeWakeUpLater,
cmdHomeWakeUpEarlier, cmdHomeGoToSleepLater, cmdHomeGoToSleepEarlier, cmdOfficeWakeUpLater,
cmdOfficeWakeUpEarlier, cmdOfficeLeaveLater, cmdOfficeLeaveEarlier, cmdOfficeComeHomeLater,
cmdOfficeComeHomeEarlier, cmdOfficeGoToSleepLater, cmdOfficeGoToSleepEarlier, cmdWeekendWakeUpLater,
cmdWeekendWakeUpEarlier, cmdWeekendGoToSleepLater, cmdWeekendGoToSleepEarlier, cmdHighTemperatureUp,
cmdHighTemperatureDown, cmdLowTemperatureUp, cmdLowTemperatureDown, cmdSensorOffsetUp,
cmdSensorOffsetDown, cmdCommandNotRecognized"""

controlCommands = [cmd.strip()[3:] for cmd in CTR.replace('\n','').split(', ')]
menuCommands = [cmd.strip()[3:] for cmd in MNU.replace('\n','').split(', ')]


import pyperclip

pyperclip.copy('\n'.join(controlCommands + ['---'] + menuCommands))

In [51]:
LIST = """cmdMenuMain
cmdSetpointHigher
cmdSetpointLower
cmdSetpointAuto
cmdGoAway
cmdUpdateStatus
cmdMenuOverruleToday
cmdMenuOverruleTomorrow
cmdMenuOverruleMultipleDays
cmdMenuSettings
cmdMenuHome
cmdOverruleTodayWorkFromHome
cmdOverruleTodayWorkAtOffice
cmdOverruleTodayWeekend
cmdOverruleTodayAway
cmdOverruleTodayAutomatic
cmdOverruleTomorrowWorkFromHome
cmdOverruleTomorrowWorkAtOffice
cmdOverruleTomorrowWeekend
cmdOverruleTomorrowAway
cmdOverruleTomorrowAutomatic
cmdOverruleMultipleMoreDays
cmdOverruleMultipleFewerDays
cmdMultipleDaySchedule
cmdOverruleMultipleWorkFromHome
cmdOverruleMultipleWorkAtOffice
cmdOverruleMultipleWeekend
cmdOverruleMultipleAway
cmdOverruleMultipleAutomatic
cmdMenuWeekSchedule
cmdMenuHomeTimes
cmdMenuOfficeTimes
cmdMenuWeekendTimes
cmdMenuTemperature
cmdMenuSensorOffset
cmdReportBoiler
cmdReportTiming
cmdMonday
cmdTuesday
cmdWednesday
cmdThursday
cmdFriday
cmdSaturday
cmdSunday
cmdSetWeekSchedule
cmdWorkFromHome
cmdWorkAtOffice
cmdWeekend
cmdAllDayAway
cmdHomeWakeUpEarlier
cmdHomeWakeUpLater
cmdHomeGoToSleepEarlier
cmdHomeGoToSleepLater
cmdOfficeWakeUpEarlier
cmdOfficeWakeUpLater
cmdOfficeLeaveEarlier
cmdOfficeLeaveLater
cmdOfficeComeHomeEarlier
cmdOfficeComeHomeLater
cmdOfficeGoToSleepEarlier
cmdOfficeGoToSleepLater
cmdWeekendWakeUpEarlier
cmdWeekendWakeUpLater
cmdWeekendGoToSleepEarlier
cmdWeekendGoToSleepLater
cmdHighTemperatureUp
cmdHighTemperatureDown
cmdLowTemperatureUp
cmdLowTemperatureDown
cmdSensorOffsetUp
cmdSensorOffsetDown
cmdSetTemperatureOffset
cmdControl
cmdCommandNotRecognized""".split('\n')

results = ''
newLine = 'enum command_t { '

for cmd in LIST:
    if len(newLine)+len(cmd)>115:
        results+='\n'
        results+=newLine
        newLine='  '
    newLine+=cmd+', '

results+='\n'+newLine

import pyperclip
pyperclip.copy(results)


In [52]:
CODE = """
    else if( messageToController.command == cmdControl                        ) control();
    else if( messageToController.command == cmdSetTemperatureOffset           ) setTemperatureOffset(messageToController.temperatureOffset);
    else if( messageToController.command == cmdSensorOffsetUp           ) increaseSensorOffset();
    else if( messageToController.command == cmdSensorOffsetDown           ) decreaseSensorOffset();
    else if( messageToController.command == cmdSetpointLower                  ) setpointLower();
    else if( messageToController.command == cmdSetpointAuto                   ) setpointAuto();
    else if( messageToController.command == cmdSetpointHigher                 ) setpointHigher();
    else if( messageToController.command == cmdGoAway                         ) goAway();
    else if( messageToController.command == cmdOverruleTodayWorkFromHome      ) overruleTodayWorkFromHome();
    else if( messageToController.command == cmdOverruleTodayWorkAtOffice    ) overruleTodayWorkFromOffice();
    else if( messageToController.command == cmdOverruleTodayWeekend           ) overruleTodayWeekend();
    else if( messageToController.command == cmdOverruleTodayAway              ) overruleTodayAway();
    else if( messageToController.command == cmdOverruleTodayAutomatic         ) overruleTodayAutomatic();
    else if( messageToController.command == cmdOverruleTomorrowWorkFromHome   ) overruleTomorrowWorkFromHome();
    else if( messageToController.command == cmdOverruleTomorrowWorkAtOffice ) overruleTomorrowWorkFromOffice();
    else if( messageToController.command == cmdOverruleTomorrowWeekend        ) overruleTomorrowWeekend();
    else if( messageToController.command == cmdOverruleTomorrowAway           ) overruleTomorrowAway();
    else if( messageToController.command == cmdOverruleTomorrowAutomatic      ) overruleTomorrowAutomatic();
    else if( messageToController.command == cmdOverruleMultipleWorkFromHome   ) overruleMultipleWorkFromHome();
    else if( messageToController.command == cmdOverruleMultipleWorkAtOffice ) overruleMultipleWorkFromOffice();
    else if( messageToController.command == cmdOverruleMultipleWeekend      ) overruleMultipleWeekend();
    else if( messageToController.command == cmdOverruleMultipleAway         ) overruleMultipleAway();
    else if( messageToController.command == cmdOverruleMultipleAutomatic    ) overruleMultipleAutomatic();
    else if( messageToController.command == cmdOverruleMultipleFewerDays    ) overruleMultipleLessDays();
    else if( messageToController.command == cmdOverruleMultipleMoreDays     ) overruleMultipleMoreDays();
    else if( messageToController.command == cmdSetWeekSchedule              ) setWeekSchedule(messageToController.dayOfWeek, messageToController.typeOfDay);
    else if( messageToController.command == cmdHomeWakeUpEarlier            ) workFromHomeWakeUpEarlier();
    else if( messageToController.command == cmdHomeWakeUpLater              ) workFromHomeWakeUpLater();
    else if( messageToController.command == cmdHomeGoToSleepEarlier         ) workFromHomeSleepEarlier();
    else if( messageToController.command == cmdHomeGoToSleepLater           ) workFromHomeSleepLater();
    else if( messageToController.command == cmdOfficeWakeUpEarlier          ) workFromOfficeWakeUpEarlier();
    else if( messageToController.command == cmdOfficeWakeUpLater            ) workFromOfficeWakeUpLater();
    else if( messageToController.command == cmdOfficeLeaveEarlier     ) workFromOfficeGoOutEarlier();
    else if( messageToController.command == cmdOfficeLeaveLater       ) workFromOfficeGoOutLater();
    else if( messageToController.command == cmdOfficeComeHomeEarlier    ) workFromOfficeComeInEarlier();
    else if( messageToController.command == cmdOfficeComeHomeLater      ) workFromOfficeComeInLater();
    else if( messageToController.command == cmdOfficeGoToSleepEarlier     ) workFromOfficeSleepEarlier();
    else if( messageToController.command == cmdOfficeGoToSleepLater       ) workFromOfficeSleepLater();
    else if( messageToController.command == cmdWeekendWakeUpEarlier           ) weekendWakeUpEarlier();
    else if( messageToController.command == cmdWeekendWakeUpLater             ) weekendWakeUpLater();
    else if( messageToController.command == cmdWeekendGoToSleepEarlier            ) weekendSleepEarlier();
    else if( messageToController.command == cmdWeekendGoToSleepLater              ) weekendSleepLater();
    else if( messageToController.command == cmdHighTemperatureDown                  ) highTempLower();
    else if( messageToController.command == cmdHighTemperatureUp                 ) highTempHigher();
    else if( messageToController.command == cmdLowTemperatureDown                   ) lowTempLower();
    else if( messageToController.command == cmdLowTemperatureUp                  ) lowTempHigher();"""

results=''
maxA=0
for line in CODE.split('\n'):
    line=line.replace('    else if( messageToController.command == ', '')
    components = [c.strip() for c in line.split(' ) ')]
    if len(components)==2:
        a,b = components
        if len(a)>maxA:
            maxA=len(a)
        a=(a+' '*100)[:32]
        results+= '  case '+a+': '+b+'\n'
    else:
        print(line)

print(maxA)
print(results[:400])
import pyperclip
pyperclip.copy(results)


31
  case cmdControl                      : control();
  case cmdSetTemperatureOffset         : setTemperatureOffset(messageToController.temperatureOffset);
  case cmdSensorOffsetUp               : increaseSensorOffset();
  case cmdSensorOffsetDown             : decreaseSensorOffset();
  case cmdSetpointLower                : setpointLower();
  case cmdSetpointAuto                 : setpointAuto();
 


In [ ]:
MENUS = """  { scnHome,                     { cmdSetpointHigher, cmdMenuMain, cmdSetpointLower } },
  { scnMain,                     { cmdMenuOverruleToday, cmdMenuOverruleTomorrow, cmdMenuOverruleMultipleDays, cmdMenuSettings, cmdMenuHome} },
  { scnOverruleToday,            { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdOverruleTodayAway, cmdOverruleTodayAutomatic, cmdMenuMain } },
  { scnOverruleTomorrow,         { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdOverruleTodayAway, cmdOverruleTodayAutomatic, cmdMenuMain } },
  { scnOverruleMultiple,         { cmdOverruleMultipleMoreDays, cmdOverruleMultipleFewerDays, cmdMultipleDaySchedule, cmdOverruleMultipleAutomatic, cmdMenuMain } },
  { scnOverruleMultipleSchedule, { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdOverruleTodayAway, cmdOverruleTodayAutomatic, cmdMenuOverruleMultipleDays } },
  { scnSettingsMain,             { cmdMenuTemperature, cmdMenuSensorOffset, cmdMenuWeekSchedule, cmdMenuHomeTimes, cmdMenuOfficeTimes, cmdMenuWeekendTimes, cmdMenuMain } },
  { scnSettingsTemperature,      { cmdHighTemperatureUp, cmdHighTemperatureDown, cmdLowTemperatureUp, cmdLowTemperatureDown, cmdMenuSettings } },
  { scnSettingsSensorOffset,     { cmdSensorOffsetUp, cmdSensorOffsetDown, cmdMenuSettings } },
  { scnSettingsWeekSchedule,     { cmdMonday, cmdTuesday, cmdWednesday, cmdThursday, cmdFriday, cmdSaturday, cmdSunday, cmdMenuSettings} },
  { scnSettingsDaySchedule,      { cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdOverruleTodayAway, cmdMenuWeekSchedule } },
  { scnSettingsHomeTimes,        { cmdHomeWakeUpLater, cmdHomeWakeUpEarlier, cmdHomeGoToSleepLater, cmdHomeGoToSleepEarlier, cmdMenuSettings } },
  { scnSettingsOfficeTimes,      { cmdOfficeWakeUpLater, cmdOfficeWakeUpEarlier, cmdOfficeLeaveLater, cmdOfficeLeaveEarlier, cmdOfficeComeHomeLater, cmdOfficeComeHomeEarlier, cmdHomeGoToSleepLater, cmdHomeGoToSleepEarlier, cmdMenuSettings } },
  { scnSettingsWeekendTimes,     { cmdWeekendWakeUpLater, cmdWeekendWakeUpEarlier, cmdWeekendGoToSleepLater, cmdWeekendGoToSleepEarlier, cmdMenuSettings } } """

cmds = []
results = []
for line in MENUS.split('\n'):
    line = line.replace('{','').replace('}','')
    items = [i.strip() for i in line.split(',')]
    results.append( '  // '+items[0] )
    for item in items[1:]:
      if item not in cmds:
        results.append('  { ' + (item +' '*30)[:30]+ ',  iconHome         },' )
        cmds.append(item)

print( max([len(c) for c in cmds]))
print('\n'.join(results[:15]))

import pyperclip
pyperclip.copy('\n'.join(results))

28
  // scnHome
  { cmdSetpointHigher             ,  iconHome         },
  { cmdMenuMain                   ,  iconHome         },
  { cmdSetpointLower              ,  iconHome         },
  {                               ,  iconHome         },
  // scnMain
  { cmdMenuOverruleToday          ,  iconHome         },
  { cmdMenuOverruleTomorrow       ,  iconHome         },
  { cmdMenuOverruleMultipleDays   ,  iconHome         },
  { cmdMenuSettings               ,  iconHome         },
  { cmdMenuHome                   ,  iconHome         },
  // scnOverruleToday
  { cmdWorkFromHome               ,  iconHome         },
  { cmdWorkAtOffice               ,  iconHome         },
  { cmdWeekend                    ,  iconHome         },


In [ ]:
# Check if all commands have a label

COMMANDS = """cmdControl, cmdSetpointHigher, cmdSetpointLower, cmdSetpointAuto, cmdGoAway, cmdOverruleTodayWorkFromHome, cmdOverruleTodayWorkAtOffice, cmdOverruleTodayWeekend, cmdOverruleTodayAway,
  cmdOverruleTodayAutomatic, cmdOverruleTomorrowWorkFromHome, cmdOverruleTomorrowWorkAtOffice, cmdOverruleTomorrowWeekend, cmdOverruleTomorrowAway, cmdOverruleTomorrowAutomatic,
  cmdOverruleMultipleMoreDays, cmdOverruleMultipleFewerDays, cmdOverruleMultipleWorkFromHome, cmdOverruleMultipleWorkAtOffice, cmdOverruleMultipleWeekend, cmdOverruleMultipleAway,
  cmdOverruleMultipleAutomatic, cmdSetWeekSchedule, cmdHomeWakeUpEarlier, cmdHomeWakeUpLater, cmdHomeGoToSleepEarlier, cmdHomeGoToSleepLater, cmdOfficeWakeUpEarlier,
  cmdOfficeWakeUpLater, cmdOfficeLeaveEarlier, cmdOfficeLeaveLater, cmdOfficeComeHomeEarlier, cmdOfficeComeHomeLater, cmdOfficeGoToSleepEarlier, cmdOfficeGoToSleepLater,
  cmdWeekendWakeUpEarlier, cmdWeekendWakeUpLater, cmdWeekendGoToSleepEarlier, cmdWeekendGoToSleepLater, cmdHighTemperatureUp, cmdHighTemperatureDown, cmdLowTemperatureUp,
  cmdLowTemperatureDown, cmdSensorOffsetUp, cmdSensorOffsetDown, cmdSetTemperatureOffset, cmdUpdateStatus, cmdUpdateScreen, cmdSilentScreenUpdate,
  cmdLastControllerCommand,
  cmdMenuMain, cmdMenuOverruleToday, cmdMenuOverruleTomorrow, cmdMenuOverruleMultipleDays, cmdMenuSettings, cmdMenuHome, cmdMultipleDaySchedule,
  cmdMenuWeekSchedule, cmdMenuHomeTimes, cmdMenuOfficeTimes, cmdMenuWeekendTimes, cmdMenuTemperature, cmdMenuSensorOffset, cmdReportBoiler, cmdReportTiming, cmdReportDebug,
  cmdMonday, cmdTuesday, cmdWednesday, cmdThursday, cmdFriday, cmdSaturday, cmdSunday, cmdWorkFromHome, cmdWorkAtOffice, cmdWeekend, cmdAllDayAway, cmdCommandNotRecognized,
  cmdBoilerSending, cmdBacklightOn, cmdKeyUp, cmdKeySelect, cmdKeyDown, cmdKeyPressed, cmdDisableTelegram, cmdEnableTelegram"""

COMMAND_LABELS = """ { cmdMenuMain                     , "Main menu"},
  { cmdSetpointHigher               , "Setpoint higher"},
  { cmdSetpointLower                , "Setpoint lower"},
  { cmdSetpointAuto                 , "Setpoint automatic"},
  { cmdGoAway                       , "Go out"},
  { cmdUpdateStatus                 , "Update"},
  { cmdMenuOverruleToday            , "Overrule today"},
  { cmdMenuOverruleTomorrow         , "Overrule tomorrow"},
  { cmdMenuOverruleMultipleDays     , "Overrule multiple days"},
  { cmdMenuSettings                 , "Settings"},
  { cmdMenuHome                     , "Home menu"},
  { cmdOverruleTodayWorkFromHome    , "Work from home"},
  { cmdOverruleTodayWorkAtOffice    , "Work at office"},
  { cmdOverruleTodayWeekend         , "Weekend"},
  { cmdOverruleTodayAway            , "All day away"},
  { cmdOverruleTodayAutomatic       , "Automatic"},
  { cmdOverruleTomorrowWorkFromHome , "Work from home"},
  { cmdOverruleTomorrowWorkAtOffice , "Work from office"},
  { cmdOverruleTomorrowWeekend      , "Weekend"},
  { cmdOverruleTomorrowAway         , "All day away"},
  { cmdOverruleTomorrowAutomatic    , "Automatic"},
  { cmdOverruleMultipleMoreDays     , "More days"},
  { cmdOverruleMultipleFewerDays    , "Fewer days"},
  { cmdMultipleDaySchedule          , "Set day schedule"},
  { cmdOverruleMultipleAutomatic    , "Automatic"},
  { cmdOverruleMultipleWorkFromHome , "Work from home"},
  { cmdOverruleMultipleWorkAtOffice , "Work from office"},
  { cmdOverruleMultipleWeekend      , "Weekend"},
  { cmdOverruleMultipleAway         , "All day away"},
  { cmdMenuWeekSchedule             , "Set weekly schedule"},
  { cmdMenuHomeTimes                , "Set home times"},
  { cmdMenuOfficeTimes              , "Set office times"},
  { cmdMenuWeekendTimes             , "Set weekend times"},
  { cmdMenuTemperature              , "Set temperatures"},
  { cmdMenuSensorOffset             , "Adjust sensor offset"},
  { cmdReportBoiler                 , "Report boiler"},
  { cmdReportTiming                 , "Report timing"},
  { cmdReportDebug                  , "Report debug"},
  { cmdMonday                       , "Monday"},
  { cmdTuesday                      , "Tuesday"},
  { cmdWednesday                    , "Wednesday"},
  { cmdThursday                     , "Thursday"},
  { cmdFriday                       , "Friday"},
  { cmdSaturday                     , "Saturday"},
  { cmdSunday                       , "Sunday"},
  { cmdWorkFromHome                 , "Work from home"},
  { cmdWorkAtOffice                 , "Work from office"},
  { cmdWeekend                      , "Weekend day"},
  { cmdAllDayAway                   , "All day away"},
  { cmdHomeWakeUpLater              , "Wake up later"},
  { cmdHomeWakeUpEarlier            , "Wake up earlier"},
  { cmdHomeGoToSleepLater           , "Go to sleep later"},
  { cmdHomeGoToSleepEarlier         , "Go to sleep earlier"},
  { cmdOfficeWakeUpLater            , "Wake up later"},
  { cmdOfficeWakeUpEarlier          , "Wake up earlier"},
  { cmdOfficeLeaveLater             , "Leave later"},
  { cmdOfficeLeaveEarlier           , "Leave earlier"},
  { cmdOfficeComeHomeLater          , "Come in later"},
  { cmdOfficeComeHomeEarlier        , "Come in earlier"},
  { cmdOfficeGoToSleepLater         , "Go to sleep later"},
  { cmdOfficeGoToSleepEarlier       , "Go to sleep earlier"},
  { cmdWeekendWakeUpLater           , "Wake up later"},
  { cmdWeekendWakeUpEarlier         , "Wake up earlier"},
  { cmdWeekendGoToSleepLater        , "Go to sleep later"},
  { cmdWeekendGoToSleepEarlier      , "Go to sleep earlier"},
  { cmdHighTemperatureUp            , "High temp up"},
  { cmdHighTemperatureDown          , "High temp down"},
  { cmdLowTemperatureUp             , "Low temp up"},
  { cmdLowTemperatureDown           , "Low temp down"},
  { cmdSensorOffsetUp               , "Offset up"},
  { cmdSensorOffsetDown             , "Offset down"},
  { cmdSetWeekSchedule              , "Set week schedule"},
  { cmdSetTemperatureOffset         , "Set temperatureoffset"},
  { cmdControl                      , "Control temperature"},
  { cmdCommandNotRecognized         , "Command not recognized"},
  { cmdBoilerSending                , "Boiler sending icon"},
  { cmdBoilerNormal                 , "Boiler normal icon"},
  { cmdBacklightOn                  , "Turn backlight on"},
  { cmdKeyUp                        , "Key up"},
  { cmdKeySelect                    , "Key select"},
  { cmdKeyDown                      , "Key down"},
  { cmdSilentScreenUpdate           , "Silent screen update"},
  { cmdUpdateScreen                 , "Update screen"},
  { cmdKeyPressed                   , "Key pressed"},
  { cmdDisableTelegram              , "Disable Telegram"},
  { cmdEnableTelegram               , "Enable Telegram"}"""

CTRL = """    case cmdControl                      : control(); break;
    case cmdUpdateScreen                 : control(); break; // First update the control data before rewriting the screen
    case cmdSetTemperatureOffset         : setTemperatureOffset(messageToController.temperatureOffset); break;
    case cmdSensorOffsetUp               : increaseSensorOffset(); break;
    case cmdSensorOffsetDown             : decreaseSensorOffset(); break;
    case cmdSetpointLower                : setpointLower(); break;
    case cmdSetpointAuto                 : setpointAuto(); break;
    case cmdSetpointHigher               : setpointHigher(); break;
    case cmdGoAway                       : goAway(); break;
    case cmdOverruleTodayWorkFromHome    : overruleTodayWorkFromHome(); break;
    case cmdOverruleTodayWorkAtOffice    : overruleTodayWorkFromOffice(); break;
    case cmdOverruleTodayWeekend         : overruleTodayWeekend(); break;
    case cmdOverruleTodayAway            : overruleTodayAway(); break;
    case cmdOverruleTodayAutomatic       : overruleTodayAutomatic(); break;
    case cmdOverruleTomorrowWorkFromHome : overruleTomorrowWorkFromHome(); break;
    case cmdOverruleTomorrowWorkAtOffice : overruleTomorrowWorkFromOffice(); break;
    case cmdOverruleTomorrowWeekend      : overruleTomorrowWeekend(); break;
    case cmdOverruleTomorrowAway         : overruleTomorrowAway(); break;
    case cmdOverruleTomorrowAutomatic    : overruleTomorrowAutomatic(); break;
    case cmdOverruleMultipleWorkFromHome : overruleMultipleWorkFromHome(); break;
    case cmdOverruleMultipleWorkAtOffice : overruleMultipleWorkFromOffice(); break;
    case cmdOverruleMultipleWeekend      : overruleMultipleWeekend(); break;
    case cmdOverruleMultipleAway         : overruleMultipleAway(); break;
    case cmdOverruleMultipleAutomatic    : overruleMultipleAutomatic(); break;
    case cmdOverruleMultipleFewerDays    : overruleMultipleLessDays(); break;
    case cmdOverruleMultipleMoreDays     : overruleMultipleMoreDays(); break;
    case cmdSetWeekSchedule              : setWeekSchedule(messageToController.dayOfWeek, messageToController.typeOfDay); break;
    case cmdHomeWakeUpEarlier            : workFromHomeWakeUpEarlier(); break;
    case cmdHomeWakeUpLater              : workFromHomeWakeUpLater(); break;
    case cmdHomeGoToSleepEarlier         : workFromHomeSleepEarlier(); break;
    case cmdHomeGoToSleepLater           : workFromHomeSleepLater(); break;
    case cmdOfficeWakeUpEarlier          : workFromOfficeWakeUpEarlier(); break;
    case cmdOfficeWakeUpLater            : workFromOfficeWakeUpLater(); break;
    case cmdOfficeLeaveEarlier           : workFromOfficeGoOutEarlier(); break;
    case cmdOfficeLeaveLater             : workFromOfficeGoOutLater(); break;
    case cmdOfficeComeHomeEarlier        : workFromOfficeComeInEarlier(); break;
    case cmdOfficeComeHomeLater          : workFromOfficeComeInLater(); break;
    case cmdOfficeGoToSleepEarlier       : workFromOfficeSleepEarlier(); break;
    case cmdOfficeGoToSleepLater         : workFromOfficeSleepLater(); break;
    case cmdWeekendWakeUpEarlier         : weekendWakeUpEarlier(); break;
    case cmdWeekendWakeUpLater           : weekendWakeUpLater(); break;
    case cmdWeekendGoToSleepEarlier      : weekendSleepEarlier(); break;
    case cmdWeekendGoToSleepLater        : weekendSleepLater(); break;
    case cmdHighTemperatureDown          : highTempLower(); break;
    case cmdHighTemperatureUp            : highTempHigher(); break;
    case cmdLowTemperatureDown           : lowTempLower(); break;
    case cmdLowTemperatureUp             : lowTempHigher(); break;"""

ctrlCommands = [cmd.split(':')[0].replace('case','').strip() for cmd in CTRL.split('\n')]

countGood=0
commandsWithLabel = []
comm_lab = []
maxCommLen = 1
for line in COMMAND_LABELS.split('\n'):
    line = line.replace('{','')
    line = line.replace('}','')
    command, label = line.split(',')[:2]
    command = command.strip()
    label = label.strip()
    if len(command)>maxCommLen:
      maxCommLen = len(command)
    comm_lab.append( (command, label))
    commandsWithLabel.append(command)

commands = [c.strip() for c in COMMANDS.split(',')]

for c in commands:
  if c not in commandsWithLabel:
    print(c, 'has no label')

results=''
for command, label in comm_lab:
  results += '  { '+ (command + maxCommLen*' ')[:maxCommLen] + ' , ' + label + '},\n'


results = ''
counter=0
for cmd in commands:
  if cmd in ctrlCommands:
    results += '{sort}\t{ID}\t{cmd}\n'.format(sort=0, ID=counter, cmd=cmd)
  else:
    results += '{sort}\t{ID}\t{cmd}\n'.format(sort=1, ID=counter, cmd=cmd)
  counter+=1;

import pyperclip
pyperclip.copy(results)

print(results)


cmdLastControllerCommand has no label
0	0	cmdControl
0	1	cmdSetpointHigher
0	2	cmdSetpointLower
0	3	cmdSetpointAuto
0	4	cmdGoAway
0	5	cmdOverruleTodayWorkFromHome
0	6	cmdOverruleTodayWorkAtOffice
0	7	cmdOverruleTodayWeekend
0	8	cmdOverruleTodayAway
0	9	cmdOverruleTodayAutomatic
0	10	cmdOverruleTomorrowWorkFromHome
0	11	cmdOverruleTomorrowWorkAtOffice
0	12	cmdOverruleTomorrowWeekend
0	13	cmdOverruleTomorrowAway
0	14	cmdOverruleTomorrowAutomatic
0	15	cmdOverruleMultipleMoreDays
0	16	cmdOverruleMultipleFewerDays
0	17	cmdOverruleMultipleWorkFromHome
0	18	cmdOverruleMultipleWorkAtOffice
0	19	cmdOverruleMultipleWeekend
0	20	cmdOverruleMultipleAway
0	21	cmdOverruleMultipleAutomatic
0	22	cmdSetWeekSchedule
0	23	cmdHomeWakeUpEarlier
0	24	cmdHomeWakeUpLater
0	25	cmdHomeGoToSleepEarlier
0	26	cmdHomeGoToSleepLater
0	27	cmdOfficeWakeUpEarlier
0	28	cmdOfficeWakeUpLater
0	29	cmdOfficeLeaveEarlier
0	30	cmdOfficeLeaveLater
0	31	cmdOfficeComeHomeEarlier
0	32	cmdOfficeComeHomeLater
0	33	cmdOfficeGoToSlee

In [ ]:
EMOTICONS = """const String EMOTICON_BATH               = "\xF0\x9F\x9B\x81";
const String EMOTICON_BED                = "\xF0\x9F\x9B\x8F";
const String EMOTICON_BULLSEYE           = "\xF0\x9F\x8E\xAF";
const String EMOTICON_CALENDAR           = "\xF0\x9F\x93\x85";
const String EMOTICON_CAMERA             = "\xF0\x9F\x93\xB7";
const String EMOTICON_CHECKERED_FLAG     = "\xF0\x9F\x8F\x81";
const String EMOTICON_CHECKMARK          = "\xE2\x9C\x85";
const String EMOTICON_CLOCK              = "\xE2\x8F\xB0";
const String EMOTICON_ALARM_CLOCK        = "\xE2\x8F\xB0";
const String EMOTICON_CROSSMARK          = "\xE2\x9D\x8C";
const String EMOTICON_DOWN_ARROW         = "\xE2\xAC\x87\xEF\xB8\x8F";
const String EMOTICON_FLAME              = "\xF0\x9F\x94\xA5";
const String EMOTICON_FOOTSTEPS          = "\xF0\x9F\x91\xA3";
const String EMOTICON_GEAR               = "\xE2\x9A\x99\xEF\xB8\x8F";
const String EMOTICON_GLASSES            = "\xF0\x9F\x91\x93";
const String EMOTICON_HAMMER_WRENCH      = "\xF0\x9F\x9B\xA0";
const String EMOTICON_HOURGLASS          = "\xE2\x8C\x9B\xEF\xB8\x8F";
const String EMOTICON_HOUSE              = "\xF0\x9F\x8F\xA0";
const String EMOTICON_ISLAND             = "\xF0\x9F\x8F\x96";
const String EMOTICON_LAMP               = "\xF0\x9F\x92\xA1";
const String EMOTICON_MAGIC_STICK        = "\xF0\x9F\xAA\x84";
const String EMOTICON_OFFICE             = "\xF0\x9F\x8F\xA2";
const String EMOTICON_PIN                = "\xF0\x9F\x93\x8D";
const String EMOTICON_POINTING_FINGER    = "\xF0\x9F\x91\x89";
const String EMOTICON_RED_QUESTION_MARK  = "\xE2\x9D\x93";
const String EMOTICON_RUNNER             = "\xF0\x9F\x8F\x83";
const String EMOTICON_SHOWER             = "\xF0\x9F\x9A\xBF";
const String EMOTICON_SMILEY             = "\xF0\x9F\x98\x80";
const String EMOTICON_STHETOSCOPE        = "\xF0\x9F\xA9\xBA";
const String EMOTICON_STOPWATCH          = "\xE2\x8F\xB1";
const String EMOTICON_THERMOMETER        = "\xF0\x9F\x8C\xA1";
const String EMOTICON_TRIANGLE_DOWN      = "\xF0\x9F\x94\xBD";
const String EMOTICON_TRIANGLE_UP        = "\xF0\x9F\x94\xBC";
const String EMOTICON_UP_ARROW           = "\xE2\xAC\x86\xEF\xB8\x8F";
const String EMOTICON_WARNING            = "\xE2\x9A\xA0\xEF\xB8\x8F";"""

results = ''
maxA=0
for line in EMOTICONS.split('\n'):
    a,b = line.split('=')
    a=a.replace('const String', '')
    a=a.strip()+'[]'
    if len(a)>maxA:
        maxA = len(a)
    a=(a+42*' ')[:30]
    b=b.replace(';','')
    b=b.strip()[1:-1]
    chars=[hex(ord(c)) for c in b]+[ hex(0) ]
    results += 'const char ' + a + ' = {' + ', '.join(chars) + '};\n'

print(maxA)
import pyperclip
pyperclip.copy(results)
print(results)

28
const char EMOTICON_BATH[]                = {0xf0, 0x9f, 0x9b, 0x81, 0x0};
const char EMOTICON_BED[]                 = {0xf0, 0x9f, 0x9b, 0x8f, 0x0};
const char EMOTICON_BULLSEYE[]            = {0xf0, 0x9f, 0x8e, 0xaf, 0x0};
const char EMOTICON_CALENDAR[]            = {0xf0, 0x9f, 0x93, 0x85, 0x0};
const char EMOTICON_CAMERA[]              = {0xf0, 0x9f, 0x93, 0xb7, 0x0};
const char EMOTICON_CHECKERED_FLAG[]      = {0xf0, 0x9f, 0x8f, 0x81, 0x0};
const char EMOTICON_CHECKMARK[]           = {0xe2, 0x9c, 0x85, 0x0};
const char EMOTICON_CLOCK[]               = {0xe2, 0x8f, 0xb0, 0x0};
const char EMOTICON_ALARM_CLOCK[]         = {0xe2, 0x8f, 0xb0, 0x0};
const char EMOTICON_CROSSMARK[]           = {0xe2, 0x9d, 0x8c, 0x0};
const char EMOTICON_DOWN_ARROW[]          = {0xe2, 0xac, 0x87, 0xef, 0xb8, 0x8f, 0x0};
const char EMOTICON_FLAME[]               = {0xf0, 0x9f, 0x94, 0xa5, 0x0};
const char EMOTICON_FOOTSTEPS[]           = {0xf0, 0x9f, 0x91, 0xa3, 0x0};
const char EMOTICON_GEAR[]        